Analisis de sentimiento del contenido de los titulares de el diario online perfil.com

* Analizar como obtener los titulares de perfil
* Detectar 
* Obtener una lista con los enlaces
* 


In [24]:
# Imports
import requests
from bs4 import BeautifulSoup
import pickle
from time import sleep
import pandas as pd 
import re
import string
from collections import Counter
import nltk
from nltk.corpus import PlaintextCorpusReader
from sklearn.feature_extraction import text 
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np
from textblob import TextBlob
import math
from gensim import matutils, models
import scipy.sparse
from nltk import word_tokenize, pos_tag
from sklearn.feature_extraction.text import CountVectorizer
import csv
from datetime import datetime


In [69]:
# indicar la ruta
#url_page = 'https://infobae.com'
url_page = 'https://www.perfil.com/biblioteca/09/2006'

In [70]:
# tarda 480 milisegundos
page = requests.get(url_page).text 
soup = BeautifulSoup(page, "lxml")
soup

<!DOCTYPE html>
<html lang="es">
<!--
FECHA: 18-11-2019 15:33:57 -0300
-->
<head>
<base href="https://www.perfil.com/"/>
<meta charset="utf-8"/>
<meta content="IE=edge" http-equiv="X-UA-Compatible"/>
<meta content="user-scalable=no, width=device-width, initial-scale=1" name="viewport"/>
<title>Biblioteca | Perfil</title>
<script src="https://code.jquery.com/jquery-1.12.0.min.js"></script>
<script type="text/javascript">var currentChannelName = ""; var currentChannelSlug = ""; </script>
<script src="static/js/app/app.js?v=1.0.110" type="text/javascript"></script>
<script src="static/js/redirect-mobile.js?v=1.0.110" type="text/javascript"></script>
<!-- Google Analytics -->
<script>
(function(i,s,o,g,r,a,m){i['GoogleAnalyticsObject']=r;i[r]=i[r]||function(){
(i[r].q=i[r].q||[]).push(arguments)},i[r].l=1*new Date();a=s.createElement(o),
m=s.getElementsByTagName(o)[0];a.async=1;a.src=g;m.parentNode.insertBefore(a,m)
})(window,document,'script','https://www.google-analytics.com/analytics.js

In [87]:
# Esta es la funcion que nos devuelve un los titulares de cada mes
def get_titulares(url_page):
    page = requests.get(url_page).text 
    soup = BeautifulSoup(page, "lxml")
    for divs in soup.find_all('div', attrs={'class': 'panel'}):
        for a in divs.find_all('a'):
            href = a.get('href')
            parr = a.p.getText()
            match = re.search('\d{8}', href)
            fecha = datetime.datetime.strptime(match.group(), '%Y%m%d').date()
            fila = [fecha.year, fecha.month, fecha.day, parr.lower()]
            #for items in fila:
            #   print(items)
            # Guardamos en un csv
            return fila


2006
9
10
córdoba: alianza delasotista se impone en marcos juarez
2006
9
10
el reclutamiento docente cae en picada
2006
9
10
relatos de una obsesión: "ya no podía entrar al aula" 
2006
9
10
el fuego en la mente de los hombres
2006
9
10
será injusticia 
2006
9
10
irak se decide en ee.uu.
2006
9
10
argenblogs
2006
9
10
divididos en dos grandes familias
2006
9
10
yabrán no era el problema
2006
9
10
pesimismo pop
2006
9
10
la chica que rompió el molde
2006
9
10
¿la decadencia del imperio?
2006
9
10
si sarmiento viviera, ¿sería maestro?
2006
9
10
nueva york espectral
2006
9
10
disparen sobre blumberg
2006
9
10
a cinco años del mega atentado
2006
9
10
tormentas en el aire
2006
9
10
sabato y alberto f, en el museo de la revolución
2006
9
10
cuota = alquiler, ¿pero en la práctica?
2006
9
10
política y economía en américa latina
2006
9
10
el terrible fantasma del "ingenieri".
2006
9
10
¿quién comenzó esta pesadilla?
2006
9
10
los dinosaurios van a desaparecer
2006
9
10
salinas será declarado pe

2006
9
20
ence despidió al 80 por ciento de su personal y estudia abandonar su proyecto en el uruguay
2006
9
20
kirchner: el país volvió al lugar que nunca debió abandonar
2006
9
20
pilotos advierten sobre un cromañón aéreo 
2006
9
20
vecinos en pie de guerra por el llamado a explotar una de las mayores reservas de oro y plata
2006
9
20
el inter ganó gracias a crespo
2006
9
20
se presentó perfil.com con un debate sobre la web, la libertad de prensa y la relación con los gobiernos 
2006
9
20
recompensa por testigo desaparecido
2006
9
20
etchecolatz recusó a un juez por tener "un hijo desaparecido"
2006
9
20
lanata, castro y biasatti, las "marcas" más confiables del país
2006
9
20
más renuncias en el entorno de lula: cayó el jefe de campaña
2006
9
20
la solicitada anti k de los bonistas norteamericanos
2006
9
20
tailandia: el vice primer ministro derrocado quiere ser titular de la onu
2006
9
20
merlo quiere cortar la racha ante quilmes
2006
9
20
la argentina venció a cuba y va por el 9

In [67]:
for link in soup.find_all('a'):
    print(link.get('href'))

https://radio.perfil.com/en-vivo/radio
https://radio.perfil.com/en-vivo/radio
https://radio.perfil.com/en-vivo/radio
https://www.facebook.com/perfilcom/
https://twitter.com/perfilcom/
https://www.instagram.com/perfilcom/
https://youtube.com/perfilcom/
None
http://kioscoperfil.com/
/
https://www.perfil.com/ultimo-momento/
https://www.perfil.com/columnistas/
https://www.perfil.com/seccion/politica/
https://www.perfil.com/seccion/economia/
https://www.perfil.com/seccion/pymes/
https://www.perfil.com/seccion/sociedad/
https://www.perfil.com/seccion/ciencia/
https://www.perfil.com/seccion/tecnologia/
https://www.perfil.com/seccion/cultura/
https://www.perfil.com/seccion/arte/
https://www.perfil.com/seccion/protagonistas/
https://www.perfil.com/seccion/educacion/
https://www.perfil.com/seccion/deportes/
#
https://www.perfil.com/seccion/espectaculos/
https://www.perfil.com/seccion/turismo/
https://www.perfil.com/seccion/video/
https://www.perfil.com/criticas/
https://www.perfil.com/seccion/co

In [68]:
import re, datetime
s = "noticias/politica/la-hipotesis-de-secuestro-de-lopez-es-cada-vez-mas-fuerte-20061201-0018.phtml"
match = re.search('\d{8}', s)
date = datetime.datetime.strptime(match.group(), '%Y%m%d').date()
print(date)

2006-12-01
